In [ ]:
import pandas as pd
import json
import numpy as np
import spacy
from spacy.training.example import Example #holds information for one training instance
import random
from tqdm import tqdm
from spacy.scorer import Scorer
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import re

In [ ]:
# Authenticate
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

# Get email of current Colab user
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
email = gcloud_tokeninfo['email']

Mounted at /content/drive


In [ ]:
# Define filepath
if email == 'tamaraexterkate93@gmail.com':
  filename = "/content/drive/MyDrive/TUe/TM/Exports/export_41675_project-41675-at-2023-10-04-09-37-9bbbec63.json"
elif email == 'n.v.diermen@student.tue.nl':
  filename = "/content/drive/MyDrive/Text Mining/export_41675_project-41675-at-2023-10-04-12-08-05f5e3f5.json"
elif email == 'silkewijdeven@gmail.com':
  filename = "/content/drive/MyDrive/master/Text-mining/export_41675_project-41675-at-2023-10-04-12-08-05f5e3f5.json"

In [ ]:
DATA = []

# read json file
with open(filename, 'rb') as fp:
  training_data = json.load(fp)

  # get text, labels, relations, benchmark for each article
  for article in training_data:
    entities = []
    relations = []
    original_text = article.get('data').get('text')
    if len(article.get('annotations')) == 5: # N=5
      benchmark = True
    else:
      benchmark = False
    for annotation in article.get('annotations'):
      if annotation.get('ground_truth') == True: # only include ground truth
        for ind, label in enumerate(annotation.get('result')):
          if label.get('type') == 'labels':
            id = label.get('id')
            start = label.get('value').get('start')
            end = label.get('value').get('end')
            label_ = label.get('value').get('labels')[0] # note: cannot deal with multiple labels
            entities.append((id, start, end, label_))
          elif label.get('type') == 'relation':
            from_id = label.get('from_id')
            to_id = label.get('to_id')
            relation = label.get('labels')[0] # note: cannot deal with multiple relations
            relations.append((from_id, to_id, relation))
          else:
            print("found unknown label type (no label or relation)")

    # append article to training data as dictionary
    DATA.append({
    'text': original_text,
    'entities': entities,
    'relations': relations,
    'benchmark': benchmark
    })

In [ ]:
# get distances between relations
all_distances = {}
for data in DATA:
  for relation in data['relations']:
    rel_from_begin = [entity[1] for entity in data['entities'] if entity[0] == relation[0]]
    rel_to_begin = [entity[1] for entity in data['entities'] if entity[0] == relation[1]]
    distance = max(rel_from_begin[0], rel_to_begin[0]) - min(rel_from_begin[0], rel_to_begin[0])

    if all_distances.get(relation[2]) == None:
      all_distances[relation[2]] = abs(distance)
    else:
      all_distances[relation[2]] = max(all_distances[relation[2]], abs(distance))
    #print("relation from word at index:", rel_from_begin, "relation to word at index:", rel_to_begin, "absolute distance between words:", abs(distance))

print(all_distances)


relation from word at index: [0] relation to word at index: [20] absolute distance between words: 20
relation from word at index: [0] relation to word at index: [34] absolute distance between words: 34
relation from word at index: [0] relation to word at index: [55] absolute distance between words: 55
relation from word at index: [0] relation to word at index: [62] absolute distance between words: 62
relation from word at index: [298] relation to word at index: [314] absolute distance between words: 16
relation from word at index: [298] relation to word at index: [292] absolute distance between words: 6
relation from word at index: [375] relation to word at index: [406] absolute distance between words: 31
relation from word at index: [375] relation to word at index: [468] absolute distance between words: 93
relation from word at index: [298] relation to word at index: [0] absolute distance between words: 298
relation from word at index: [375] relation to word at index: [0] absolute dis